In [22]:
from langchain_core.tools import tool
import pandas as pd
import plotly.express as px

from typing import Annotated, Dict, List
import random

df = pd.DataFrame(
    {
        "Year": [2018, 2019, 2020, 2021, 2022, 2023],
        "GDP (Billion USD)": [2851.41, 2697.81, 3141.51, None, None, None],
    }
)


def draw_chart(df: pd.DataFrame):
    fig = px.line(df, x="Year", y="GDP (Billion USD)", title="UK GDP from 2018 to 2023")
    fig.show()


@tool
def draw_chart_tool(
    df_dict: Annotated[
        Dict[str, List[int]],
        "This dictionary has keys of 'Year' and 'GDP (Billion USD)' as list",
    ]
):
    """
    This is used to draw a chart from a dictionary of data.
    """
    df = pd.DataFrame(df_dict)
    return draw_chart(df)


@tool
def data_generator_tool():
    """
    This is used to generate a sample data for a chart.
    """
    df = pd.DataFrame(
        {
            "Year": [x for x in range(2018, 2024)],
            "GDP (Billion USD)": [random.randint(1000, 10000) for _ in range(6)],
        }
    )
    return df

In [ ]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

load_dotenv()

In [13]:
import functools

from langchain_core.messages import AIMessage


# Helper function to create a node for a given agent
def agent_node(state, agent, name):
    result = agent.invoke(state)
    # We convert the agent output into a format that is suitable to append to the global state
    if isinstance(result, ToolMessage):
        pass
    else:
        result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        # Since we have a strict workflow, we can
        # track the sender so we know who to pass to next.
        "sender": name,
    }

In [12]:
llm = ChatOpenAI(model="gpt-4o")

In [16]:
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import END, StateGraph, START


def create_agent(llm, tools, system_message: str):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You are a helpful AI assistant, collaborating with other assistants."
                " Use the provided tools to progress towards answering the question."
                " If you are unable to fully answer, that's OK, another assistant with different tools "
                " will help where you left off. Execute what you can to make progress."
                " If you or any of the other assistants have the final answer or deliverable,"
                " prefix your response with FINAL ANSWER so the team knows to stop."
                " You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(system_message=system_message)
    prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm.bind_tools(tools)

In [26]:
# data_generator
data_generator_agent = create_agent(
    llm,
    [data_generator_tool],
    system_message="Any data you generate will be visible by the user.",
)

# chart_generator
chart_agent = create_agent(
    llm,
    [draw_chart_tool],
    system_message="Any charts you display will be visible by the user.",
)
chart_node = functools.partial(agent_node, agent=chart_agent, name="chart_generator")

In [21]:
import operator
from typing import Annotated, Sequence
from typing_extensions import TypedDict

from langchain_openai import ChatOpenAI


# This defines the object that is passed between each node
# in the graph. We will create different nodes for each agent and tool
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    sender: str

In [23]:
from langgraph.prebuilt import ToolNode

tools = [data_generator_tool, draw_chart_tool]
tool_node = ToolNode(tools)

In [24]:
# Either agent can decide to end
from typing import Literal


def router(state):
    # This is the router
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        # The previous agent is invoking a tool
        return "call_tool"
    if "FINAL ANSWER" in last_message.content:
        # Any agent decided the work is done
        return END
    return "continue"

In [28]:
workflow = StateGraph(AgentState)
workflow.add_node("chart_generator", chart_node)
workflow.add_node("data_generator", data_generator_agent)
workflow.add_node("call_node", tool_node)

workflow.add_conditional_edges(
    "call_node",
    lambda x: x["sender"],
    {
        "data_generator": "data_generator",
        "chart_generator": "chart_generator",
    },
)
workflow.add_edge(START, "data_generator")
graph = workflow.compile()

ValueError: Node `call_node` is not reachable

In [18]:
chart_node.invoke(
    {
        "messages": [
            HumanMessage(content="Generate a sample data for a chart and show it")
        ]
    }
)

AttributeError: 'functools.partial' object has no attribute 'invoke'